In [2]:
from pathlib import Path
import re

import pandas as pd
import numpy as np

from dandi.dandiapi import DandiAPIClient

DANDISET_ID = '000409'

client = DandiAPIClient()
dandiset = client.get_dandiset(DANDISET_ID, 'draft')
meta_data = dandiset.get_metadata()

assets = list(dandiset.get_assets())

# I'm going to build a dataframe containing the asset paths and filesizes so I can navigate and filter the data files
df_assets = pd.DataFrame(index=np.arange(len(assets)), columns=['path', 'eid', 'subject', 'filesize', 'name'])

for i, asset in enumerate(assets):
    parts = asset.path.split('/')
    # gets what is after 'sub-' in the first element of parts
    subject = re.search('sub-(.*)', parts[0]).group(1)
    eid = re.search(f'sub-{subject}_ses-(.*)', parts[1]).group(1)[:36]
    df_assets.loc[i, 'subject'] = subject
    df_assets.loc[i, 'eid'] = eid
    df_assets.loc[i, 'path'] = asset.path
    df_assets.loc[i, 'filesize'] = asset.size
    df_assets.loc[i, 'name'] = Path(asset.path).name
df_assets = df_assets.set_index('eid')
df_assets.head()

,path,subject,filesize,name
eid,,,,
ae8787b1-4229-4d56-b0c2-566b61a25b77,sub-NR-0027/sub-NR-0027_ses-ae8787b1-4229-4d56...,NR-0027,1209097503,sub-NR-0027_ses-ae8787b1-4229-4d56-b0c2-566b61...
ae8787b1-4229-4d56-b0c2-566b61a25b77,sub-NR-0027/sub-NR-0027_ses-ae8787b1-4229-4d56...,NR-0027,7173043026,sub-NR-0027_ses-ae8787b1-4229-4d56-b0c2-566b61...
ae8787b1-4229-4d56-b0c2-566b61a25b77,sub-NR-0027/sub-NR-0027_ses-ae8787b1-4229-4d56...,NR-0027,9430320168,sub-NR-0027_ses-ae8787b1-4229-4d56-b0c2-566b61...
1d4a7bd6-296a-48b9-b20e-bd0ac80750a5,sub-PL015/sub-PL015_ses-1d4a7bd6-296a-48b9-b20...,PL015,1394782933,sub-PL015_ses-1d4a7bd6-296a-48b9-b20e-bd0ac807...
1d4a7bd6-296a-48b9-b20e-bd0ac80750a5,sub-PL015/sub-PL015_ses-1d4a7bd6-296a-48b9-b20...,PL015,5918819563,sub-PL015_ses-1d4a7bd6-296a-48b9-b20e-bd0ac807...


In [4]:
# pick the experiment ID from https://viz.internationalbrainlab.org/
eid = "64e3fb86-928c-4079-865c-b364205b502e"
print(df_assets.loc[eid, 'path'].values)

['sub-NYU-46/sub-NYU-46_ses-64e3fb86-928c-4079-865c-b364205b502e_behavior+ecephys+image/sub-NYU-46_ses-64e3fb86-928c-4079-865c-b364205b502e_OriginalVideoBodyCamera.mp4'
 'sub-NYU-46/sub-NYU-46_ses-64e3fb86-928c-4079-865c-b364205b502e_behavior+ecephys+image/sub-NYU-46_ses-64e3fb86-928c-4079-865c-b364205b502e_OriginalVideoRightCamera.mp4'
 'sub-NYU-46/sub-NYU-46_ses-64e3fb86-928c-4079-865c-b364205b502e_behavior+ecephys+image/sub-NYU-46_ses-64e3fb86-928c-4079-865c-b364205b502e_OriginalVideoLeftCamera.mp4'
 'sub-NYU-46/sub-NYU-46_ses-64e3fb86-928c-4079-865c-b364205b502e_behavior+ecephys+image.nwb'
 'sub-NYU-46/sub-NYU-46_ses-64e3fb86-928c-4079-865c-b364205b502e-processed-only_behavior.nwb']


In [22]:
%%capture output
!pip install one-api
from one.api import ONE
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')

In [50]:
brain_acronym = 'SSs5'
# query sessions endpoint
_, insertions = one.search_insertions(atlas_acronym=brain_acronym, details=True)
sessions = list(set([ins['session'] for ins in insertions]))

# print the name of assets that contain the desired brain regions
print(f'No. of detected sessions: {len(sessions)}')
for _, asset in df_assets.loc[sessions].iterrows():
    if 'only_behavior' in asset.path:
        print(asset.path)


No. of detected sessions: 7
sub-KS094/sub-KS094_ses-3f859b5c-e73a-4044-b49e-34bb81e96715-processed-only_behavior.nwb
sub-KS043/sub-KS043_ses-e5c75b62-6871-4135-b3d0-f6464c2d90c0-processed-only_behavior.nwb
sub-NYU-46/sub-NYU-46_ses-64e3fb86-928c-4079-865c-b364205b502e-processed-only_behavior.nwb
sub-ibl-witten-29/sub-ibl-witten-29_ses-821f1883-27f3-411d-afd3-fb8241bbc39a-processed-only_behavior.nwb
sub-CSH-ZAD-025/sub-CSH-ZAD-025_ses-ff48aa1d-ef30-4903-ac34-8c41b738c1b9-processed-only_behavior.nwb
sub-SWC-043/sub-SWC-043_ses-6fb1e12c-883b-46d1-a745-473cde3232c8-processed-only_behavior.nwb
sub-KS046/sub-KS046_ses-0ac8d013-b91e-4732-bc7b-a1164ff3e445-processed-only_behavior.nwb
